In [2]:
import pandas as pd
import numpy as np
from scipy.fft import fft2, fftshift
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [112]:
df = pd.read_csv('/Users/varunchowdary/Desktop/DSLFinalProject/dsl_final_project_ai_vs_real_art/detect ai artwork - resizing images for efficiency/frequency_analysis/image_fft_features.csv')

X = df.drop('Label', axis=1)
y = df['Label']




3410


In [113]:
df

for col in df.columns.tolist():
    print("Column" + col + str(df[col].isna().sum()))
from sklearn.impute import SimpleImputer





ColumnMean0
ColumnStd0
ColumnSkewness0
ColumnKurtosis0
ColumnEnergy0
ColumnEntropy74
ColumnLabel0
2728


In [114]:
import time
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

start_time = time.time()  # Start timing
rf_model = RandomForestClassifier(criterion='entropy', random_state=42)
rf_model.fit(X_train_imputed, y_train)
end_time = time.time()  # End timing

y_pred = rf_model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

# Calculate runtime in milliseconds
runtime_ms = (end_time - start_time) * 1000
print(f"Runtime: {runtime_ms:.2f} milliseconds")

Accuracy: 0.8167155425219942
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       328
           1       0.86      0.77      0.81       354

    accuracy                           0.82       682
   macro avg       0.82      0.82      0.82       682
weighted avg       0.82      0.82      0.82       682

Runtime: 528.03 milliseconds


Basic Random Forest

Accuracy: 0.8167155425219942
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       328
           1       0.86      0.77      0.81       354

    accuracy                           0.82       682
   macro avg       0.82      0.82      0.82       682
weighted avg       0.82      0.82      0.82       682



Parameter Tuning

In [11]:
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None, 10,],
}


grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy'), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_imputed, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'max_depth': None, 'n_estimators': 300}
Best score: 0.8082850775920082


In [12]:
best_model = grid_search.best_estimator_
y_pred_opt = best_model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred_opt)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred_opt))

Accuracy: 0.8079178885630498
              precision    recall  f1-score   support

           0       0.77      0.86      0.81       328
           1       0.85      0.76      0.80       354

    accuracy                           0.81       682
   macro avg       0.81      0.81      0.81       682
weighted avg       0.81      0.81      0.81       682



Support Vector Machines

This might work because all columns are numerical. It doesnt really work that well.

In [17]:
from sklearn.svm import SVC


Below will be the code for NN

In [90]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        
        return x

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [91]:
# Define hyperparameters
input_size = X_train_imputed.shape[1]
hidden_size1 = 100
hidden_size2 = 50
output_size = 2  # Assuming binary classification

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create the model
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)



# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print(torch.Tensor.size(X_train_tensor))
print(y_train.size)

torch.Size([2728, 6])
2728


In [124]:
# Training the model
import matplotlib.pyplot as plt

losses = []
val_losses = []

start_time = time.time()  # Start timing

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())


# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(classification_report(y_test_tensor.numpy(), predicted.numpy()))
    print("Accuracy:", accuracy)
    val_loss = criterion(outputs, y_test_tensor)
    val_losses.append(val_loss.item())

end_time = time.time()  # Start timing


runtime_ms = (end_time - start_time) * 1000
print(f"Runtime: {runtime_ms:.2f} milliseconds")


              precision    recall  f1-score   support

           0       0.78      0.84      0.81       328
           1       0.84      0.78      0.81       354

    accuracy                           0.81       682
   macro avg       0.81      0.81      0.81       682
weighted avg       0.81      0.81      0.81       682

Accuracy: 0.8064516129032258
Runtime: 114.01 milliseconds


Catboost stuff

In [99]:
from catboost import CatBoostClassifier


In [126]:
start_time = time.time()  # Start timing

clf = CatBoostClassifier(loss_function='Logloss', random_seed=42)
clf.fit(X_train_scaled, y_train)
end_time = time.time()  # Start timing


y_pred = clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

runtime_ms = (end_time - start_time) * 1000
print(f"Runtime: {runtime_ms:.2f} milliseconds")

Learning rate set to 0.015814
0:	learn: 0.6853752	total: 10.6ms	remaining: 10.6s
1:	learn: 0.6780750	total: 13.8ms	remaining: 6.87s
2:	learn: 0.6709103	total: 16.3ms	remaining: 5.4s
3:	learn: 0.6632126	total: 17.7ms	remaining: 4.4s
4:	learn: 0.6560980	total: 19.6ms	remaining: 3.9s
5:	learn: 0.6499071	total: 22.2ms	remaining: 3.68s
6:	learn: 0.6435804	total: 24.8ms	remaining: 3.51s
7:	learn: 0.6366394	total: 26ms	remaining: 3.23s
8:	learn: 0.6310529	total: 28.5ms	remaining: 3.14s
9:	learn: 0.6254618	total: 31.9ms	remaining: 3.15s
10:	learn: 0.6197784	total: 34ms	remaining: 3.06s
11:	learn: 0.6139626	total: 36.1ms	remaining: 2.97s
12:	learn: 0.6082298	total: 38.7ms	remaining: 2.94s
13:	learn: 0.6031338	total: 40.2ms	remaining: 2.83s
14:	learn: 0.5978377	total: 42ms	remaining: 2.76s
15:	learn: 0.5926112	total: 43.6ms	remaining: 2.68s
16:	learn: 0.5877764	total: 45ms	remaining: 2.6s
17:	learn: 0.5831540	total: 46.3ms	remaining: 2.52s
18:	learn: 0.5785049	total: 47.7ms	remaining: 2.46s
19:	

Accuracy: 0.8123167155425219
              precision    recall  f1-score   support

           0       0.76      0.88      0.82       328
           1       0.87      0.75      0.80       354

    accuracy                           0.81       682
   macro avg       0.82      0.81      0.81       682
weighted avg       0.82      0.81      0.81       682



In [103]:
param_grid = {
    'iterations': [100, 200, 300],  # Number of trees (boosting iterations)
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
    'depth': [4, 6, 8],  # Depth of the trees
    'l2_leaf_reg': [1, 3, 5],  # L2 regularization coefficient
}

grid_search = GridSearchCV(CatBoostClassifier(loss_function='Logloss', random_seed=42), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

0:	learn: 0.6904457	total: 1.12ms	remaining: 111ms
1:	learn: 0.6875207	total: 1.96ms	remaining: 95.9ms
2:	learn: 0.6848997	total: 2.5ms	remaining: 80.9ms
3:	learn: 0.6824442	total: 3.13ms	remaining: 75.1ms
4:	learn: 0.6797098	total: 3.75ms	remaining: 71.3ms
5:	learn: 0.6770844	total: 4.41ms	remaining: 69.1ms
6:	learn: 0.6740884	total: 4.87ms	remaining: 64.7ms
7:	learn: 0.6715432	total: 5.34ms	remaining: 61.4ms
8:	learn: 0.6689683	total: 5.85ms	remaining: 59.1ms
9:	learn: 0.6661912	total: 6.22ms	remaining: 56ms
10:	learn: 0.6636776	total: 6.75ms	remaining: 54.6ms
11:	learn: 0.6612756	total: 7.42ms	remaining: 54.4ms
12:	learn: 0.6590244	total: 9.13ms	remaining: 61.1ms
13:	learn: 0.6568270	total: 10.3ms	remaining: 63.1ms
14:	learn: 0.6543092	total: 10.7ms	remaining: 60.6ms
15:	learn: 0.6520926	total: 11.1ms	remaining: 58.4ms
16:	learn: 0.6498336	total: 11.8ms	remaining: 57.5ms
17:	learn: 0.6475408	total: 13.1ms	remaining: 59.8ms
18:	learn: 0.6454555	total: 14.9ms	remaining: 63.4ms
19:	lea

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x15e9e7f70>,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'l2_leaf_reg': [1, 3, 5],
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='accuracy')

In [105]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 3, 'learning_rate': 0.05}
Best score: 0.8108503890682108


In [106]:
best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
y_pred_opt = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_opt)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred_opt))

0:	learn: 0.6691745	total: 17.2ms	remaining: 3.43s
1:	learn: 0.6463395	total: 20.5ms	remaining: 2.03s
2:	learn: 0.6272168	total: 22ms	remaining: 1.44s
3:	learn: 0.6079615	total: 25.4ms	remaining: 1.24s
4:	learn: 0.5909544	total: 29.9ms	remaining: 1.16s
5:	learn: 0.5774471	total: 33.5ms	remaining: 1.08s
6:	learn: 0.5649420	total: 36.4ms	remaining: 1s
7:	learn: 0.5519491	total: 38.3ms	remaining: 918ms
8:	learn: 0.5434715	total: 40.6ms	remaining: 861ms
9:	learn: 0.5328961	total: 43.3ms	remaining: 822ms
10:	learn: 0.5231197	total: 47.1ms	remaining: 809ms
11:	learn: 0.5135536	total: 49.8ms	remaining: 780ms
12:	learn: 0.5059210	total: 52.7ms	remaining: 759ms
13:	learn: 0.4989225	total: 54.7ms	remaining: 726ms
14:	learn: 0.4919710	total: 56.7ms	remaining: 700ms
15:	learn: 0.4860004	total: 58.7ms	remaining: 675ms
16:	learn: 0.4809563	total: 60.3ms	remaining: 649ms
17:	learn: 0.4747491	total: 61.7ms	remaining: 624ms
18:	learn: 0.4700780	total: 64.3ms	remaining: 612ms
19:	learn: 0.4655504	total:

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Predictions
y_pred = log_reg.predict(X_test_scaled)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.78      0.73       328
           1       0.77      0.68      0.72       354

    accuracy                           0.73       682
   macro avg       0.73      0.73      0.73       682
weighted avg       0.73      0.73      0.73       682

